In [8]:
%reload_ext autoreload
%autoreload 2
import sys
from dotenv import load_dotenv
import os
load_dotenv()

ROOT = os.getenv("ROOT")
sys.path.append(ROOT)

from rich.progress import track
from transformers import AutoTokenizer
from src.nlp.regressor import BertRegressorPipeline, ReviewsDataset
from src.data_loader import load_reviews
from src.processing import process_reviews

In [4]:
df = load_reviews(category="All_beauty", frac=0.001)
df = process_reviews(df=df, clean_text=False)
sub = df.rename(columns={"rating": "target", "review_input": "text"})[["target", "text"]]

[12:32:17] | INFO  | loaded 702 rows                                                              ]8;id=163277;file:///Users/lucanyckees/Desktop/my-repos/recommendation_system/src/data_loader.py\data_loader.py]8;;\:]8;id=523620;file:///Users/lucanyckees/Desktop/my-repos/recommendation_system/src/data_loader.py#18\18]8;;\

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-uncased",
    max_length=512,
    truncation=True,
    clean_up_tokenization_spaces=False,
)

In [13]:
count = 0

for index, r in track(sub.iterrows(), description="tokenizing"):
    tokens = tokenizer.tokenize(r["text"])
    tokens = ["[CLS]"] + tokens + ["[SEP]"]
    if len(tokens) < 512:
        tokens = tokens + ["[PAD]" for _ in range(512 - len(tokens))]
    else:
        count += 1
        print(f"len(tokens): {len(tokens)}, updating tokens")
        tokens = tokens[: 512 - 1] + ["[SEP]"]
        print(f">>> len(tokens): {len(tokens)}")
        
print(count)

/Users/lucanyckees/Desktop/my-repos/recommendation_system/venv/lib/python3.11/site-packages/rich/live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

len(tokens): 628, updating tokens

>>> len(tokens): 512

len(tokens): 646, updating tokens

>>> len(tokens): 512

len(tokens): 1082, updating tokens

>>> len(tokens): 512

3
